# CNN Encoder + RNN Time Series Prediction Model with Limit Orderbook Data

## Read About Timedistributed Layer
https://machinelearningmastery.com/timedistributed-layer-for-long-short-term-memory-networks-in-python/

### Requirements
- Implement Your Own CNN Encoder +  LSTM Model for Trinary Classification Model

### Data Download at: https://drive.google.com/open?id=1_GXzTuyIopvkkOeCxHanVZKa0tKclD6F

# Data Preprocessing


#### Procedures
- Preparation
  1. Unzip Data
  2. Configurations

- Preprocessing
  1. Import Data
  2. Create Balanced Panel
  3. Create Target Variable
  4. Scale Data
  5. Train / Test Split
  6. Create Sequences
 

### Data Description
- There are two files, 1. "coin_price_dfs.csv" and 2. "orderbook_dfs.csv"

1. coin_price_dfs.csv
  - This dataset includes price (high, low, start, last, volume) for 1 minute interval
  - columns
    -  timestamp: trading time
    - currency: type of currency
    - price
      - high
      - last
      - low
      - start
    - volume: trading volume
    
2. orderbook_dfs.csv
 - This dataset includes order book information for 1 minute interval
 - columns
    - timestamp: trading time
    - ask_price: the lowest price for selling order
    - ask_strength [0~99]: each bin contains the volume of ask order in between - 0.1% x (num) ~ - 0.1% x (num+1) from  last price
    - bid_price: the highest price for buying order
    - bid_stregth [0~99]: each bin contains the volume of bid order in between - 0.1% x (num) ~ - 0.1% x (num+1) from  last price
    - currency: type of currency
    - bid_ask_spread: spread between ask and bid price

## Preparation

### 1. Data Import and Create Balanced Panel

In [0]:
%matplotlib inline

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

In [0]:
DATA_PATH = "/content/gdrive/My Drive/Lecture/StudyPie/Data/CoinOne/"

In [0]:
!ls "/content/gdrive/My Drive/Lecture/StudyPie/Data/CoinOne/"

In [0]:
# Unzip Data
# It will take more than 5 min
import zipfile
import io

zf = zipfile.ZipFile(DATA_PATH+"orderbook_data.zip", "r")
zf.extractall(DATA_PATH)

### 2. Configurations

In [0]:
currency_types = ["btc","xrp","eth","bch"]
currency_targets = ["btc"]

TIME_FREQUENCY = 1 # 1, 2, 3, 30 minutes
TIME_SEQ_LEN = 60
FUTURE_PERIOD_PREDICT = 5

# NUM_DEPENDENT_VARIABLE == 2: increase/decrease
# NUM_DEPENDENT_VARIABLE == 3: incrase/no change/decrease
NUM_DEPENDENT_VARIABLE = 3
SIGNIFICANT_CRITERIA = 0.3 # for no change criteria

BATCH_SIZE = 128
NUM_ITERATIONS = 30

TRAIN_RATIO = 0.8

## Preprocessing

### 1. Import Data

In [0]:
import pandas as pd

# read coin price data
coin_price_dfs = pd.read_csv(DATA_PATH+"coin_price_dfs.csv")
# read orderbeook data
orderbook_dfs = pd.read_csv(DATA_PATH+"orderbook_dfs.csv")    

# sort by timestamp
coin_price_dfs = coin_price_dfs.sort_values(by=['timestamp','currency'])
orderbook_dfs = orderbook_dfs.sort_values(by=['timestamp','currency'])    

# select the columns of interests
coin_price_dfs = coin_price_dfs[coin_price_dfs["currency"].isin(currency_types)]
orderbook_dfs = orderbook_dfs[orderbook_dfs["currency"].isin(currency_types)]

print(coin_price_dfs.shape)
print(orderbook_dfs.shape)

### 2. Create Balanced Panel

In [0]:
coinprice_features = ['currency','last','volume','timestamp']

# we only use bid / ask volume data inbetween current price and 3% from it (-3%,-2.9%, =2.8% ...0%, 0.1%, 0.2%... 3%)
bid_cols = ["bid_strength_"+str(i) for i in range(30)]
ask_cols = ["ask_strength_"+str(i) for i in range(30)]
orderbook_features = bid_cols + ask_cols + ["currency","timestamp","bid_ask_spread"]

key_features = coinprice_features + orderbook_features

# merge price and orderbook data based on timestamp
panel_df = pd.merge_asof(left=coin_price_dfs[coinprice_features].reset_index(drop=True), 
                         right=orderbook_dfs[orderbook_features].reset_index(drop=True),
                         on = u'timestamp', by = u'currency')
## interpolation
panel_df = panel_df.groupby('currency').apply(lambda gp: gp.fillna(method="ffill"))
panel_df.dropna(inplace=True)

In [0]:
panel_df[panel_df['currency']=='btc'].head(n=5)

In [0]:
# our dataset is not balanced yey
# we need to transform dataframe into (timestamp - coin1 - coin2 - coin3 ...)
new_panel_df_list = []

for currency_type in currency_types:  # begin iteration
  new_col_names = {key_feature:currency_type+"_"+key_feature 
                   for key_feature in key_features if key_feature != "timestamp"}

  single_currency_panel_df = panel_df[panel_df['currency']==currency_type]
  single_currency_panel_df.rename(columns=new_col_names, inplace=True)

  print(currency_type, " size: ", single_currency_panel_df.shape[0])    
  new_panel_df_list.append(single_currency_panel_df)

In [0]:
balanced_panel_df = new_panel_df_list[0]

for new_panel_df in new_panel_df_list[1:]:
  balanced_panel_df = pd.merge_asof(left=balanced_panel_df.reset_index(drop=True), 
                       right=new_panel_df.reset_index(drop=True),
                       on = u'timestamp')

balanced_panel_df.dropna(inplace=True)    
balanced_panel_df.reset_index(drop=True,inplace=True)

In [0]:
balanced_panel_df.head()

###   3. Create Target Variable

In [0]:
# to create target variable we create return 
for currency_target in currency_targets:
  balanced_panel_df[currency_target+'_last_target'] = balanced_panel_df[currency_target+'_last'].shift(-FUTURE_PERIOD_PREDICT )
  balanced_panel_df[currency_target+'_last_target_return'] = (balanced_panel_df[currency_target+'_last_target'] 
                                                              - balanced_panel_df[currency_target+'_last'])/balanced_panel_df[currency_target+'_last']

In [0]:
def classify_trinary(values):
  gp_std = np.std(values)

  target = []
  for value in values:
    if SIGNIFICANT_CRITERIA*gp_std < value:
      target.append(2)
    elif -SIGNIFICANT_CRITERIA*gp_std > value:
      target.append(0)  
    else:
      target.append(1)  

  return target

In [0]:
import numpy as np

for currency_target in currency_targets:
  if NUM_DEPENDENT_VARIABLE == 2:
    print("NUM_DEPENDENT_VARIABLE: ", 2)
    balanced_panel_df[currency_target+'_target'] = balanced_panel_df[currency_target+'_last_target_return'].apply(lambda x: 0 if x <= 0 else 1)

  elif NUM_DEPENDENT_VARIABLE == 3:    
    print("NUM_DEPENDENT_VARIABLE: ", 3)
    print("SIGNIFICANT_CRITERIA:", SIGNIFICANT_CRITERIA)
    balanced_panel_df[currency_target+'_target'] = balanced_panel_df[currency_target+"_last_target_return"].transform(classify_trinary)    

  else:
    print("NUM_DEPENDENT_VARIABLE is not properly specified")

### 4. Scale Data
  5. Create Sequences
  6. Train / Test Split

  

- Training / Predicting Model
  1. Model Build
  2. Model Train
  3. Prediction
  4. Evaluation <- this assignment implement it

In [0]:
# we scaled input data using mix max scaler
from sklearn import preprocessing 

exclude_cols = [] # if you want to ignore some columns for mix max scaling, write the column here
exceptions = ["currency", "timestamp", "target"]

for col in balanced_panel_df.columns:
  exception_occur = False

  for exception in exceptions:
    if exception in col:
      exception_occur = True
      exclude_cols.append(col)

  if exception_occur:
    continue
  
  balanced_panel_df[col] = preprocessing.minmax_scale(balanced_panel_df[col].values)

In [0]:
# we sholud exclude these variable
# since we should not predict based on the future information
target_cols = [currency_target+"_target" for currency_target in currency_targets]

for target_col in target_cols:
    exclude_cols.remove(target_col) 

balanced_panel_df = balanced_panel_df.filter(regex="^(?!({0})$).*$".format('|'.join(exclude_cols)))

In [0]:
balanced_panel_df.head()

### 5. Train / Test Split

In [0]:
# split train test dataset
panel_df_train = balanced_panel_df.iloc[:int(balanced_panel_df.shape[0]*TRAIN_RATIO)]
panel_df_test = balanced_panel_df[~balanced_panel_df.index.isin(panel_df_train.index)]

### 6. Create Sequences

In [0]:
import random
from collections import deque

def sequence_generator(panel_df, TIME_SEQ_LEN, suffle=True,seed=101):

  sequential_data = []  # this is a list that will CONTAIN the sequences
  queue = deque(maxlen = TIME_SEQ_LEN)  # These will be our actual sequences. They are made with deque, which keeps the maximum length by popping out older values as new ones come in

  for i in panel_df.values:  # iterate over the values
    queue.append([n for n in i[:-1]])  # store all but the target
    if len(queue) == TIME_SEQ_LEN:  # make sure we have 60 sequences!
      sequential_data.append([np.array(queue), i[-1]])  # append those bad boys!

  if suffle == True:
    random.seed(seed)
    random.shuffle(sequential_data)  # shuffle for good measure.

  X = []
  y = []

  for seq, target in sequential_data:  # going over our new sequential data
    X.append(seq)  # X is the sequences
    y.append(target)  # y is the targets/labels (buys vs sell/notbuy)

  return np.array(X), y  # return X and y...and make X a numpy array!

In [0]:
train_X, train_Y = sequence_generator(panel_df_train , TIME_SEQ_LEN, suffle=True, seed=101)
test_X, test_Y = sequence_generator(panel_df_test, TIME_SEQ_LEN, suffle=False)

In [0]:
print(panel_df_train.shape)
print(train_X.shape)
print(len(train_Y))

print(panel_df_test.shape)
print(test_X.shape)
print(len(test_Y))

# Exercise CNN Encoder + RNN Time Series Prediction Model with Limit Orderbook Data

- Training / Predicting Model
  1. Model Define
    - CNN Encoder + RNN Model
  2. Model Train
  3. Prediction
  4. Evaluation <- this assignment implement it

### 1. Model Build

In [0]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM, CuDNNLSTM, BatchNormalization, Flatten

def cnn_encoder_lstm(input_dim, output_dim):
    
  """
  Write Code Here!!!
  
  Implemenent 1D Conv along the time dimension  
  
  Refer our Hands-on-Labs code
  """
  
  return model

In [0]:
model_dict = {}
model_dict["CNN_ENCODER_LSTM"] = cnn_encoder_lstm(train_X.shape[1:], NUM_DEPENDENT_VARIABLE)

### 2. Model Train


In [0]:
BATCH_SIZE = 64 
NUM_ITERATIONS = 10

hist = {}

for name, model in model_dict.items():

  print("===== Model Name: {} =====".format(name))    
  if name == "CNN_ENCODER_LSTM":
    hist[name] = model.fit(train_X.reshape(-1, train_X.shape[1:][0], train_X.shape[1:][1]),
                          tf.keras.utils.to_categorical(train_Y, num_classes=None), 
                          batch_size = BATCH_SIZE,
                          validation_split=0.2,
                          epochs = NUM_ITERATIONS)

  print("===== Train Done =====".format(name))

### 3. Prediction

In [0]:
predictions = {}

for name, model in model_dict.items():  
  print("===== Model Name: {} =====".format(name))  
  if name == "CNN_ENCODER_LSTM":
    predictions[name] = model.predict_classes(test_X.reshape(-1, train_X.shape[1:][0], train_X.shape[1:][1]))

  else:
    predictions[name] = model.predict_classes(test_X)

### 4. Evaluation

1. Comparing Accuracy both Train and Validation set data
  - Compare loss according to the below materials
  - Visualize training history  
  - Check genelarization of your model
  - Refer
    - https://medium.com/machine-learning-world/neural-networks-for-algorithmic-trading-1-2-correct-time-series-forecasting-backtesting-9776bfd9e589
  - Visualization Hint
    - https://machinelearningmastery.com/display-deep-learning-model-training-history-in-keras/

In [0]:
import matplotlib.pyplot as plt

plt.title('model loss')
legend_names = []
# summarize history for accuracy
for name, model_hist in hist.items():
  plt.plot(model_hist.history['loss'])
  plt.plot(model_hist.history['val_loss'])
  legend_names.extend([name+'_train', name+'_validation'])
  
plt.legend(legend_names, loc='upper left')

plt.ylabel('loss')
plt.xlabel('epoch')
plt.show()

In [0]:
import matplotlib.pyplot as plt

plt.title('model acc')
legend_names = []
# summarize history for accuracy
for name, model_hist in hist.items():
  plt.plot(model_hist.history['acc'])
  plt.plot(model_hist.history['val_acc'])
  legend_names.extend([name+'_train', name+'_validation'])
  
plt.legend(legend_names, loc='upper left')

plt.ylabel('acc')
plt.xlabel('epoch')
plt.show()

2. Measure Model Accuracy for Classification Problem
  - Now we are going to evaluate our model 
  - Accuracy, Recall, F1 Score based on Confusion Matrix
  - Refer definition of each scores
    - Confusion matrix https://en.wikipedia.org/wiki/Confusion_matrix
    - Confusion matrix in Korean https://datascienceschool.net/view-notebook/731e0d2ef52c41c686ba53dcaf346f32/
    
  - Hint
    - http://scikit-learn.org/stable/modules/generated/sklearn.metrics.confusion_matrix.html
    - http://scikit-learn.org/stable/modules/generated/sklearn.metrics.classification_report.html

In [0]:
from sklearn.metrics import classification_report

for name, prediction_model in predictions.items():
  print("===== Model Name: {} =====".format(name))  
  print(classification_report(prediction_model, test_Y))

3. Measure Test Set Return based on the Simplest Strategy
  - Condition
    - Initial budget = 100
 - Strategy
    - If we predict up, then buy or hold (if we already bought)
    - If we predict down or no change then sell (if we already bought) or do nothing 
  - Draw your return

In [0]:
for name, prediction_model in predictions.items():
  print("===== Model Name: {} =====".format(name))  
  
  for currency_target in currency_targets:
      bechmark_return = 100
      bechmark_return_history = []

      prediction_return = 100
      prediction_return_history = []

      buy_price = 0
      sell_price = 0

      hold = False

      panel_df_test[currency_target+'_last_return'] = panel_df_test[currency_target+'_last'].pct_change()
      
      for prediction, (i, r) in zip(prediction_model, panel_df_test.iloc[TIME_SEQ_LEN-1:].iterrows()):

          if hold == True:
              prediction_return = prediction_return*(1+r[currency_target+'_last_return'])  

          bechmark_return = bechmark_return*(1+r[currency_target+'_last_return'])
          bechmark_return_history.append(bechmark_return)
          prediction_return_history.append(prediction_return)

          if prediction ==2:
              hold = True
          else: 
              hold = False

      plt.title(currency_target)
      plt.plot(prediction_return_history, label = name+' prediction return')    
      plt.plot(bechmark_return_history, label = 'becnmark return')
      plt.xticks(rotation=30)
      plt.legend()
      plt.show()